    LOADIING DATA

In [23]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\vinays\Downloads\Health_Data_Analysis_Report.pdf")

docs = loader.load()

print(len(docs))

print(docs[0].page_content)
print(docs[1].metadata)

3
Health Data Analysis and Hypothesis Testing Report
1. Introduction
This project involves a comprehensive analysis of a health-related dataset containing 100,000 records and 49
features. The aim is to investigate the impact of various lifestyle and physiological factors on health status
(target variable). The outcome of the project is to derive data-driven insights and present them through
statistical tests and visual storytelling using Power BI.
2. Data Loading and Initial Overview
Loaded a CSV dataset with 100,000 entries and 49 features.
Features include physiological parameters (e.g., heart rate, glucose, BMI), lifestyle indicators (e.g., sleep,
screen time, alcohol consumption), and the target variable (target) indicating health status (1 = healthy, 0 =
unhealthy).
3. Data Preprocessing
Performed several critical steps to prepare the data for analysis:
- Handling Missing Values: Ensured all features had complete data by checking and imputing missing values
if needed.
- Encoding C

SPLITTING DATA

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=25)

chunks = text_splitter.split_documents(docs)

print(len(chunks))
print(chunks[100])

122
page_content='- Work hours had a statistically significant' metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250727121815', 'source': 'C:\\Users\\vinays\\Downloads\\Health_Data_Analysis_Report.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}


In [36]:
chunks[0].page_content

'Health Data Analysis and Hypothesis Testing Report'

In [37]:
for i in chunks:
    print(i.page_content)

Health Data Analysis and Hypothesis Testing Report
1. Introduction
This project involves a comprehensive analysis of
analysis of a health-related dataset containing
dataset containing 100,000 records and 49
features. The aim is to investigate the impact of
the impact of various lifestyle and physiological
and physiological factors on health status
(target variable). The outcome of the project is
of the project is to derive data-driven insights
data-driven insights and present them through
statistical tests and visual storytelling using
storytelling using Power BI.
2. Data Loading and Initial Overview
Loaded a CSV dataset with 100,000 entries and 49
100,000 entries and 49 features.
Features include physiological parameters (e.g.,
parameters (e.g., heart rate, glucose, BMI),
rate, glucose, BMI), lifestyle indicators (e.g.,
indicators (e.g., sleep,
screen time, alcohol consumption), and the target
and the target variable (target) indicating
(target) indicating health status (1 = healthy, 

In [38]:
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

embeddings = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-MiniLM-L6-v2",  
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_Api_KEY")
)

texts=[doc.page_content for doc in chunks]

vectors=embeddings.embed_documents(texts)

print(len(vectors))

print(len(vectors[0][:10]))

122
10


In [39]:

from langchain_community.vectorstores import FAISS

# Build FAISS vector store directly from Documents
vector_store = FAISS.from_documents(chunks, embeddings)

# Run similarity search
results = vector_store.similarity_search("Explain hypothesis testing", k=3)

print("\nTop matches:")
for r in results:
    print("-", r.page_content)



Top matches:
- addition to hypothesis testing.
- 4. Hypothesis Formulation and Statistical Testing
- Test Used     P-value    Conclusion


In [42]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [43]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEndpointEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025BC7593340>, search_kwargs={'k': 4})

In [44]:
retriever.invoke('What is testing')

[Document(id='8e17f482-9469-48aa-9698-3a5760e9f8ec', metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250727121815', 'source': 'C:\\Users\\vinays\\Downloads\\Health_Data_Analysis_Report.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3'}, page_content='addition to hypothesis testing.'),
 Document(id='a609475c-0e2e-48a7-ab95-2570a09a1163', metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250727121815', 'source': 'C:\\Users\\vinays\\Downloads\\Health_Data_Analysis_Report.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}, page_content='Tested individual features against the health'),
 Document(id='04449b0d-2cc9-484b-8124-15742a710ef8', metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250727121815', 'source': 'C:\\Users\\vinays\\Downloads\\Health_Data_Analysis_Report.pdf', 'total_pages': 3, 'page': 1, 'page

In [45]:
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEndpoint
import os

# Load Hugging Face LLM (example: flan-t5-base)
llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-base",
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_KEY")
)



In [47]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [70]:
question          = "how stress imposes on health"
retrieved_docs    = retriever.invoke(question)

In [71]:
retrieved_docs

[Document(id='b9a3c039-94ec-4888-a685-4d9ea716a348', metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250727121815', 'source': 'C:\\Users\\vinays\\Downloads\\Health_Data_Analysis_Report.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}, page_content='Sleep, Stress, Diet, and Physical Activity are'),
 Document(id='b4b4544c-d46b-43f3-a018-f1c0d3fc741f', metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250727121815', 'source': 'C:\\Users\\vinays\\Downloads\\Health_Data_Analysis_Report.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}, page_content='Stress Level (using logistic regression)'),
 Document(id='f63ee567-f0db-4681-9e49-b265fae753e0', metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250727121815', 'source': 'C:\\Users\\vinays\\Downloads\\Health_Data_Analysis_Report.pdf', 'total_pages': 3, 'page'

In [72]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

'Sleep, Stress, Diet, and Physical Activity are\n\nStress Level (using logistic regression)\n\nand physiological factors on health status\n\nlike sleep quality, stress level, and physical'

In [73]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [95]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      Sleep, Stress, Diet, and Physical Activity are\n\nStress Level (using logistic regression)\n\nand physiological factors on health status\n\nlike sleep quality, stress level, and physical\n      Question: how stress imposes on health\n    ")

In [121]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
import os
from dotenv import load_dotenv

load_dotenv()

llm=HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.1-Terminus",
    task="text-generation",
    
    
)
model=ChatHuggingFace(llm=llm)

res=model.invoke(final_prompt)

print(res.content)


Based on the provided transcript, the context is insufficient to explain how stress imposes on health. It only mentions "stress level" as a factor alongside sleep and diet, and refers to physiological factors affecting health status, but does not elaborate on the specific mechanisms or impacts of stress.


In [97]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [98]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [99]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [113]:
parallel_chain.invoke('stress')

{'context': 'Stress Level (using logistic regression)\n\nSleep, Stress, Diet, and Physical Activity are\n\nsummary showed that Sleep, Stress, Diet, and\n\n- Sleep Hours, Screen Time, Stress Level (using',
 'question': 'stress'}

In [114]:
parser = StrOutputParser()

In [118]:

main_chain = parallel_chain | prompt | model | parser

In [119]:
main_chain.invoke("stress")

'Based on the provided context, the term "Stress Level" is mentioned in relation to being measured using logistic regression. The context also groups it with other factors: Sleep, Diet, and Physical Activity.\n\nNo further details about what "stress" specifically refers to are available in the transcript.'